# Sytax Tree Builder
## Version 2
### This is an notebook to research all properties of syntax trees, including:
- Finding association rules

- Filling in the blank on missing word classes

- Building trees

### Version 2 will feature:
- JSON for all data structures

- Proper code decomposition

In [ ]:
#Imports
import re
import nltk
import math
import requests
import ast
from bs4 import BeautifulSoup
from PyDictionary import PyDictionary

# X-Bar and Array Tree Data Structures

In [ ]:
### Getting parts of speach using wordsmyth, for words with many meanings
word = "glass"
url = "https://www.wordsmyth.net/?level=3&ent={}".format(word)
cert = './wordsmyth.pem'

s = requests.Session()

res = s.get(url, verify = False)

soup = BeautifulSoup(res.text, 'html5lib')
mainlist = soup.findAll("dl", {"class": "mainlist"})
if len(mainlist) == 1:
    many_parts_of_speech_title = mainlist[0].findAll("dt", text='parts of speech:')
    if len(mainlist) == 1:
        many_parts_of_speech = many_parts_of_speech_title[0].findNext("dd")
        print([_.text for _ in mpos.findAll("a")])
    
maintable = soup.findAll("table", {"class": "maintable"})
if len(maintable) == 1:
    # There is a single definition for this word
    part_of_speach_row = maintable[0].findAll("tr", {"class": "postitle"})
    part_of_speach = part_of_speach_row[0].find_all('td', {"class": "data"})[0].a.text
    print(part_of_speach)

foobar = soup.findAll("div", {"class": "list_title"})
if len(foobar) == 1 and foobar[0].text == "Did you mean this word?":
    print("These are suggestions")
# This is a page to pick between multiple definitions
wordlist = soup.findAll("div", {"class": "wordlist"})
if len(wordlist) == 1:
    ref_links = wordlist[0].table.tbody.findAll("a")
    [print(ref['href']) for ref in ref_links]

In [ ]:
def saveDict(d, filename):
    sD = str(d)
    f = open(filename, 'w')
    f.write(sD)
    f.close()
    
def loadDict(filename):
    f = open(filename, 'r')
    sD = f.read()
    f.close()
    d = ast.literal_eval(sD)
    return d

In [ ]:
localBank = loadDict('local_bank.json')

In [173]:
localBank = {k:list(set(v if type(v) == type([]) else [v])) for k,v in localBank.items()}

In [371]:
saveDict(localBank, 'local_bank.json')

In [379]:
def fetchLikelyClass(word = None, rid_url = None):
    url = "https://www.wordsmyth.net/?level=3&ent={}".format(word) if word else rid_url
    s = requests.Session()
    res = s.get(url, verify = False)
    
    likely_class = None
    soup = BeautifulSoup(res.text, 'html5lib')
    
    # There are multiple parts of speech of the word, and just one definition
    '''
    mainlist = soup.findAll("dl", {"class": "mainlist"})
    if len(mainlist) == 1:
        many_parts_of_speech_title = mainlist[0].findAll("dt", text='parts of speech:')
        if len(many_parts_of_speech_title) == 1:
            many_parts_of_speech = many_parts_of_speech_title[0].findNext("dd")
            return [_.text for _ in mpos.findAll("a")]
    '''
    
    # There is a single definition for this word
    maintable = soup.findAll("table", {"class": "maintable"})
    if len(maintable) == 1:
        part_of_speach_row = maintable[0].findAll("tr", {"class": "postitle"})
        likely_class = part_of_speach_row[0].find_all('td', {"class": "data"})[0].a.text
        return [likely_class] if word else likely_class
   
    # Make sure this is not a words suggestion page, if it is return None
    foobar = soup.findAll("div", {"class": "list_title"})
    if len(foobar) == 1 and foobar[0].text.strip() == "Did you mean this word?":
        return None

    # This is a page to pick between multiple definitions
    wordlist = soup.findAll("div", {"class": "wordlist"})
    if len(wordlist) == 1:
        ref_links = wordlist[0].table.tbody.findAll("a")
        return [fetchLikelyClass(rid_url = ref['href']) for ref in ref_links]
    
    # No definition found
    return None

def getLikelyClass(word, convert = True, save = False):
    if word in localBank:
        return localBank[word]
    
    rawClassList = fetchLikelyClass(word)
    classList = list(set(rawClassList)) #[item for sublist in rawClassList for item in sublist]
    
    switch = {
        'determiner'          :        'd',
        'noun'                :        'n',
        'pronoun'             :        'pn',
        'preposition'         :        'p',
        'verb'                :        'v',
        'transitive verb'     :        'v', #vt
        'intransitive verb'   :        'v', #vi
        'ditransitive verb'   :        'v', #vd
        'adjective'           :        'adj',
        'adverb'              :        'adv',
        'conjunction'         :        'conj',
    }
    
    if convert or save:
        classList = [switch[_] for _ in classList]
    if save and classList is not None:
        localBank[word] = classList
    return classList

def defineLikelyClass(word, wordClass, force=False):
    if word not in localBank or force:
        localBank[word] = wordClass
        return True
    return False

In [ ]:
defineLikelyClass('\'s', 'd', True)
getLikelyClass('\'s', True)

In [161]:
print(getLikelyClass('catch', save = True))

['v']


In [ ]:
#For machine gunning words into the local word bank!
x = 'The dog is in the house'.split()
y = 'd n v p d n'.split()
for i in range(len(x)):
    defineLikelyClass(x[i], y[i], True)

In [195]:
'''
sentence = "The coach will be using a new approach"
'''
sentence = "The tall egg is cold"
#'''

In [288]:
#This will build all of the possible base cases for perspective
#based on research forensics for each given word
def constructPerspective(tokens, parts_of_speach):
    return [{'pos' : parts_of_speach[i], 'words' : [tokens[i]]} for i in range(len(tokens))]

def constructPerspectivePermutations(pool, pile = []):
    if len(pool) == 0:
        return pile
    else:
        if len(pile) == 0:
            return recurse(pool[1:], [[_] for _ in pool[0]])
        front_pos = pool[0]
        for pool_item in front_pos:
            for pile_item in pile:
                newpile_item = pile_item + [pool_item]
                newpile.append(newpile_item)
        return recurse(pool[1:], newpile)
    if len(newpile) > 0:
        return newpile
    return pile

def constructBasePerspectives(sentence):
    perspectives = []
    tokens = sentence.strip().split(" ")
    parts_of_speach = [getLikelyClass(_) for _ in tokens]
    permutations = constructPerspectivePermutations(parts_of_speach)
    return [constructPerspective(tokens, _) for _ in permutations]

x = constructBasePerspectives(sentence)

In [289]:
x

[[{'pos': 'd', 'words': ['The']},
  {'pos': 'adj', 'words': ['tall']},
  {'pos': 'n', 'words': ['egg']},
  {'pos': 'v', 'words': ['is']},
  {'pos': 'adj', 'words': ['cold']}],
 [{'pos': 'd', 'words': ['The']},
  {'pos': 'adj', 'words': ['tall']},
  {'pos': 'v', 'words': ['egg']},
  {'pos': 'v', 'words': ['is']},
  {'pos': 'adj', 'words': ['cold']}]]

In [290]:
#I am Michael, I love Raquel, I eat protien, etc...
rules = {
    "adjP" : [["adjbar"]],
    "adjbar" : [["adj"], ["advP", "adjbar"]],
    "advP" : [["advbar"]],
    "advbar" : [["adv"], ["advP", "advbar"]],
    "nbar": [["adjP", "nbar"], ["nbar", "pP"], ["n"], ["n", "pP"], ["pn"]],
    "nP" : [["nbar"], ["nP", "conj", "nP"]],
    "dbar" : [["d", "nP"], ["nP"]],
    "dP" : [["dP", "dbar"], ["dbar"], ["dP", "conj", "dP"]],
    "pbar" : [["p", "dP"]],
    "pP" : [["pbar"]],
    "vP" : [["vbar"], ["vP", "conj", "vP"]],
    "vbar" : [["vbar", "pP"], ["vbar", "advP"], ["advP", "vbar"], ["vbar", "dP"], ["v"], ["v", "pP"], ["v", "dP"], ["v", "adjP"], ["v", "vP"]],
    "negbar" : [["neg", "vP"]],
    "negP" : [["negbar"]],
    "t" : [["v"]],
    "tbar" : [["vP"], ["t", "vP"], ["t", "negP"]],
    "tP" : [["dP", "tbar"], ["tP", "conj", "tP"]],
    "cbar" : [["c", "tP"], ["tP"]],
    "cP" : [["cbar"], ["cP", "conj", "cP"]]
}

In [291]:
#determines if the given rule is at the top of a given tree
#Strict disallows the tree from having different neighbors at the top
def inTreeTop(tree, rule, strict = False):
    retVal = False
    strictVal = True
    for topNode in tree:
        if topNode['pos'] == rule:
            retVal = True
        else:
            strictVal = False
    if strict:
        return (retVal, strictVal)
    return retVal

In [292]:
def pruneTrees(trees):
    pruned = []
    for tree in trees:
        finishedVal = inTreeTop(tree, "cP", strict = True)
        if finishedVal[0] and finishedVal[1]:
            pruned.append(tree)
    return pruned

In [293]:
#Debugging handling, small but out of the tree builder to keep things clean
def dprint(statement, currentLevel, evalLevel, **kwargs):
    if currentLevel >= evalLevel or currentLevel == -1:
        print(statement, **kwargs)

In [294]:
def matchRules(perspective = [], verbose = 0):
    global rules
    
    matches = {}
    for rule, conditionSet in rules.items():  
        dprint("rule {}".format(rule), verbose, 5)
        for conditions in conditionSet:
            conditionMatch, conditionPosition = 0, 0
            dprint("\tcond {}".format(str(conditions)), verbose, 5, end='')
            dprint(": ", verbose, 6, end='\n')
            for i in range(len(perspective)):
                node = perspective[i]
                if node['pos'] == conditions[conditionMatch]:
                    dprint("\t\t{} <- {}({})".format(rule, node['pos'], node['words']), verbose, 6)
                    if conditionMatch == 0:
                        conditionPosition = i
                    conditionMatch += 1
                    if conditionMatch == len(conditions):
                        dprint("\t\t\trule match.", verbose, 6)
                        if rule in matches:
                            matches[rule] += [(conditionPosition, conditionPosition + conditionMatch)]
                        else:
                            matches.update({rule:[(conditionPosition, conditionPosition + conditionMatch)]})
                        conditionMatch = 0
                else:
                    conditionMatch = 0
            dprint("", verbose, 5)
    return matches

In [295]:
def generateNewPerspectives(perspective, verbose):
    new_perspectives = []
    matches = matchRules(perspective, verbose)
    dprint("Inputs to Matches:\n {} -> {}".format(perspective, matches), verbose, 2)

    dprint("Rule matches: ", verbose, 3)
    for matchRule, matchIdxTuples in matches.items():
        for match in matchIdxTuples:
            dprint("{} <- {}".format(matchRule, match), verbose, 3)
            
            subset = perspective[match[0]:match[1]]
            left, right = perspective[:match[0]], perspective[match[1]:]
            dprint("{}, {}, {}".format(left, subset, right), verbose, 4)
            new_perspective = left + [{'pos' : matchRule, 'words' : subset}] + right

            if new_perspective not in new_perspectives:
                new_perspectives.append(new_perspective)
                
    return new_perspectives

    

In [296]:
#This is the big boy itself
#The rule above define what the builder will try
#As of right now, the builder has a few problems
#It refuses to accept verb complements
#It does not account for null items
#As a result it doesn't yet generate movement
#
#Verboseness is for debugging purposes, it works in levels
#0 - Print nothing, smooth, user experience
#1 - Prints things like how many trees are being considered in any one permutation of rules
#2 - ...
#3 - ...
#... - ...
#-1 - PRINTS. EVERYTHING. Be careful what you wish for!!!
def BuildSyntaxTree(perspectives, maxNode = "cP", verbose = 0):
    #Keep track of where matches for each rule occurs, when creating a new node,
    #select the simplest matches first (adjbar before tbar), and the match that occurs later in the tree
    
    #This is temporary for now -- in the future we will have multiple perspectives, not just one!
    #perspectives = [x]
    new_perspectives, final_perspectives = [], []
    exhaustive_perspectives = perspectives.copy()
    
    #Measures tree formation
    while True:
        new_perspectives = []
        for perspective in perspectives:
            new_perspectives += generateNewPerspectives(perspective, verbose)
        dprint("new perspectives: {}".format(new_perspectives), verbose, 4)
        
        dprint("Filling in Exaustive and Final arrays...", verbose, 2)
        next_perspectives = []
        for perspective in new_perspectives:
            if perspective not in exhaustive_perspectives:
                dprint("{} -> exhaustive_perspectives, generically missing".format(perspective), verbose, 3)
                exhaustive_perspectives.append(perspective)
                next_perspectives.append(perspective)
            if inTreeTop(perspective, maxNode, strict = True) == (True, True) and perspective not in final_perspectives:
                dprint("{} -> final_perspectives, completed tree".format(perspective), verbose, 3)
                final_perspectives.append(perspective)
        
        #Stops searching
        if perspectives == next_perspectives or len(next_perspectives) == 0:
            dprint("Halting, onboarding last trees", verbose, 2)
            for perspective in next_perspectives:
                if perspective not in final_perspectives:
                    dprint("{} -> final_perspectives".format(perspective), verbose, 3)
                    final_perspectives.append(perspectives)
            dprint("Halted", verbose, 0)
            break
        perspectives = next_perspectives.copy()
        dprint("New perspectives: {}\tTotal: {}".format(len(perspectives), len(exhaustive_perspectives)), verbose, 1)
    #Returns a tuple of:
    #Final list of highest level trees
    #All generated trees
    #Changes, corresponding to exhaustive indicies
    return (final_perspectives, exhaustive_perspectives)

In [297]:
print(sentence)
p = constructBasePerspectives(sentence)
treeData = BuildSyntaxTree(p, verbose = 0)

The tall egg is cold
Halted


In [298]:
finalTree[0]

[{'pos': 'cP',
  'words': [{'pos': 'cbar',
    'words': [{'pos': 'tP',
      'words': [{'pos': 'dP',
        'words': [{'pos': 'dbar',
          'words': [{'pos': 'd', 'words': 'The'},
           {'pos': 'nP',
            'words': [{'pos': 'nbar',
              'words': [{'pos': 'adjP',
                'words': [{'pos': 'adjbar',
                  'words': [{'pos': 'adj', 'words': 'tall'}]}]},
               {'pos': 'nbar',
                'words': [{'pos': 'n', 'words': 'egg'}]}]}]}]}]},
       {'pos': 'tbar',
        'words': [{'pos': 'vP',
          'words': [{'pos': 'vbar',
            'words': [{'pos': 'v', 'words': 'is'},
             {'pos': 'adjP',
              'words': [{'pos': 'adjbar',
                'words': [{'pos': 'adj', 'words': 'cold'}]}]}]}]}]}]}]}]}]

In [299]:
finalTree = pruneTrees(treeData[0])

In [300]:
#Gives the height of a tree, for those from perspective
def treeHeight(tree):
    if type(tree) == str:
        return 0
    if len(tree) == 0:
        return 0
    if len(tree) == 1:
        if type(tree[0]) == str:
            return 0
    return 1+max([treeHeight(_['words']) for _ in tree])
    #return max([(0 if len(_) == 1 else max(treeHeight(__) for __ in _[1]) if (type(_) == list) else 0) for _ in tree]) + 1

#i = -10
#print(final_perspectives[0])#(perspectives[i])
#treeHeight(final_perspectives[0])#(perspectives[i])

In [301]:
print("Trees: {}".format(len(finalTree)))
print(treeHeight(finalTree[0]))
finalTree[0]

Trees: 1
10


[{'pos': 'cP',
  'words': [{'pos': 'cbar',
    'words': [{'pos': 'tP',
      'words': [{'pos': 'dP',
        'words': [{'pos': 'dbar',
          'words': [{'pos': 'd', 'words': ['The']},
           {'pos': 'nP',
            'words': [{'pos': 'nbar',
              'words': [{'pos': 'adjP',
                'words': [{'pos': 'adjbar',
                  'words': [{'pos': 'adj', 'words': ['tall']}]}]},
               {'pos': 'nbar',
                'words': [{'pos': 'n', 'words': ['egg']}]}]}]}]}]},
       {'pos': 'tbar',
        'words': [{'pos': 'vP',
          'words': [{'pos': 'vbar',
            'words': [{'pos': 'v', 'words': ['is']},
             {'pos': 'adjP',
              'words': [{'pos': 'adjbar',
                'words': [{'pos': 'adj', 'words': ['cold']}]}]}]}]}]}]}]}]}]

In [316]:
latexKeys = {
    "xbar" : '$\\bar{{{}}}$'
}

def latexTree(tree, depth = 1):
    if type(tree) == str:
        return tree + " "
    if type(tree) == type([]):
        t = tree[0]
        if type(t) == str:
            return t + " "
    label = t['pos']
    if len(label) == 1:
        label = label.upper()
    ploc = label.find('P')
    if ploc > 0:
        if len(label[:ploc]) == 1:
            label = label.upper()
    barloc = label.find('bar')
    if barloc > 0:
        sublabel = label[:barloc]
        if len(sublabel) == 1:
            sublabel = sublabel.upper()
        label = latexKeys["xbar"].format(sublabel)
    if len(t['words']) == 1:
        return '[.{} {}]'.format(label, latexTree(t['words'], depth+1))
    return '[.{} \n{}{} \n{}{}]'.format(label, ' '*depth, latexTree([t['words'][0]], depth+1), ' '*depth, latexTree([t['words'][1]], depth+1))
    
print(latexTree(treeData[0][0]))

[.CP [.$\bar{C}$ [.TP 
   [.DP [.$\bar{D}$ 
     [.D The ] 
     [.NP [.$\bar{N}$ 
       [.adjP [.$\bar{adj}$ [.adj tall ]]] 
       [.$\bar{N}$ [.N egg ]]]]]] 
   [.$\bar{T}$ [.VP [.$\bar{V}$ 
      [.V is ] 
      [.adjP [.$\bar{adj}$ [.adj cold ]]]]]]]]]


In [318]:
def eteTree(tree, depth = 1):
    if type(tree) == str:
        return tree + " "
    if type(tree) == type([]):
        t = tree[0]
        if type(t) == str:
            return t + " "
    label = t['pos']
    if len(label) == 1:
        label = label.upper()
    #ploc = label.find('P')
    #if ploc > 0:
    #    if len(label[:ploc]) == 1:
    #        label = label.upper()
    #barloc = label.find('bar')
    #if barloc > 0:
    #    sublabel = label[:barloc]
    #    if len(sublabel) == 1:
    #        sublabel = sublabel.upper()
    #    label = latexKeys["xbar"].format(sublabel)
    if len(t['words']) == 1:
        return '({},({}))'.format(label, eteTree(t['words'], depth+1))
    return '({},({},{}))'.format(label, eteTree([t['words'][0]], depth+1), eteTree([t['words'][1]], depth+1))
    
print(eteTree(treeData[0][0]))

(cP,((cbar,((tP,((dP,((dbar,((D,(The )),(nP,((nbar,((adjP,((adjbar,((adj,(tall )))))),(nbar,((N,(egg )))))))))))),(tbar,((vP,((vbar,((V,(is )),(adjP,((adjbar,((adj,(cold ))))))))))))))))))


In [327]:
from ete3 import Tree, TreeStyle
#t = Tree( "{};".format(eteTree(treeData[0][0])) )
ts = TreeStyle()
ts.show_leaf_name = True
ts.rotation = 90
ts.scale = 120
t.render("etetree.png", tree_style=ts)

{'nodes': [[127.0, 0.5, 131.0, 4.5, 0, None],
  [127.0, 123.5, 131.0, 127.5, 1, None],
  [127.0, 246.5, 131.0, 250.5, 2, None]],
 'faces': [[121.5, 250.0, 136.5, 259.0, 2, 'a']],
 'node_areas': {0: [121.5, 1.0, 136.5, 259.0],
  1: [121.5, 4.0, 136.5, 259.0],
  2: [121.5, 127.0, 136.5, 259.0]}}

In [349]:
def anyTree(tree, parent = None):
    if parent is None and len(tree) == 1:
        root = Node(tree[0]['pos'])
        anyTree(tree[0]['words'], root)
        return root
    for child in tree:
        if type(child) is dict:
            node = Node(child['pos'], parent = parent)
            anyTree(child['words'], node)
        if type(child) is str:
            node = Node(child, parent = parent)
    return None
    
anyTreeRoot = anyTree(treeData[0][0])

In [350]:
treeData[0][0][0]['words'][0]

{'pos': 'cbar',
 'words': [{'pos': 'tP',
   'words': [{'pos': 'dP',
     'words': [{'pos': 'dbar',
       'words': [{'pos': 'd', 'words': ['The']},
        {'pos': 'nP',
         'words': [{'pos': 'nbar',
           'words': [{'pos': 'adjP',
             'words': [{'pos': 'adjbar',
               'words': [{'pos': 'adj', 'words': ['tall']}]}]},
            {'pos': 'nbar', 'words': [{'pos': 'n', 'words': ['egg']}]}]}]}]}]},
    {'pos': 'tbar',
     'words': [{'pos': 'vP',
       'words': [{'pos': 'vbar',
         'words': [{'pos': 'v', 'words': ['is']},
          {'pos': 'adjP',
           'words': [{'pos': 'adjbar',
             'words': [{'pos': 'adj', 'words': ['cold']}]}]}]}]}]}]}]}

In [382]:
for pre, fill, node in RenderTree(anyTreeRoot):
    print("%s%s" % (pre, node.name))

cP
└── cbar
    └── tP
        ├── dP
        │   └── dbar
        │       ├── d
        │       │   └── The
        │       └── nP
        │           └── nbar
        │               └── n
        │                   └── chicken
        └── tbar
            └── vP
                └── vbar
                    └── v
                        └── ran


In [377]:
from anytree.exporter import UniqueDotExporter
#sudo apt install graphviz
def graphAnyTree(anyTreeRoot, filename):
    def nodenamefunc(node):
        return '%s' % (node.name)

    UniqueDotExporter(anyTreeRoot, 
                      nodenamefunc=nodenamefunc,
                      nodeattrfunc=lambda node: "shape=none").to_picture("{}.png".format(filename))

In [396]:
root = Node("test",display_name="test")
node = Node("test2", parent = root, display_name="test2")
node2 = Node("test3", parent = root, display_name="test2")
for line in UniqueDotExporter(root):  # doctest: +SKIP
    print(line)
UniqueDotExporter(root, 
                      nodenamefunc=nodenamefunc,
                      nodeattrfunc=lambda node: 'label="{}";shape=none'.format(node.display_name)).to_picture("{}.png".format("test"))

digraph tree {
    "0x7f40717d6208" [label="test"];
    "0x7f40717d6240" [label="test2"];
    "0x7f40717d62b0" [label="test3"];
    "0x7f40717d6208" -> "0x7f40717d6240";
    "0x7f40717d6208" -> "0x7f40717d62b0";
}


In [398]:
root.id

AttributeError: 'Node' object has no attribute 'id'

In [380]:
#A whole test from top to bottom
sentence = "The chicken ran"
p = constructBasePerspectives(sentence)
treeData = BuildSyntaxTree(p, verbose = 0)
finalTrees = pruneTrees(treeData[0])
anyTreeRoot = anyTree(finalTrees[0])
graphAnyTree(anyTreeRoot, "test")

/home/mkeays/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/home/mkeays/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/home/mkeays/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/home/mkeays/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:857: Insecu

Halted
